## Download the data

A dataset could be created and downloaded using the new [views feature](https://docs.qcarchive.molssi.org/user_guide/datasets/caching.html).

Alternatively, download live from QCArchive (see [Retrieving results](https://docs.openforcefield.org/projects/qcsubmit/en/stable/examples/retrieving-results.html) for more).

In [1]:
from collections import defaultdict
from openff.units import unit
import numpy as np
import torch
import tqdm

In [2]:
from qcportal import PortalClient

qc_client = PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")

In [3]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

In [4]:
# # Pull down the torsion drive records from a dataset.
# torsion_drive_result_collection = TorsionDriveResultCollection.from_server(
#     client=qc_client,
#     # small example dataset -- downloading and interacting with a dataset
#     # can take a while!
#     datasets="OpenFF Cresset Additional Coverage TorsionDrives v4.0",
#     spec_name="default",
# )

## Convert the data into a smee/descent-friendly format

The data needs to be postprocessed into a useful format for smee. Note, a lot of the functions here will benefit from parallelism as they can be very slow, some examples are provided in [Josh's repo](https://github.com/jthorton/SPICE-SMEE/blob/main/fit-v1).

In [5]:
# import descent.targets.energy

# bohr_to_angstrom = (1 * unit.bohr).m_as(unit.angstrom)
# hartree_to_kcal = (1 * unit.hartree * unit.avogadro_constant).m_as(
#     unit.kilocalories_per_mole
# )
# hartree_to_kcal

Ok we now want to create an [energy.Entry](https://simonboothroyd.github.io/descent/latest/reference/targets/energy/#descent.targets.energy.Entry) to be processed into a dataset. The final output is a [Huggingface dataset](https://huggingface.co/docs/datasets/en/index).

How to do this will differ for optimizations and torsiondrives slightly due to the structure of the code.
Here we look at torsiondrives, code for optimizations should be very similar but not need the `minimum_optimizations` part.

In [6]:
# # create a dict to hold data by SMILES so conformers are mostly mapped together
# # note: a more robust solution would use Molecule.are_isomorphic or similar method,
# # but here we lazily just compare the smiles strings

# data_by_smiles = defaultdict(list)
# records_and_molecules = list(torsion_drive_result_collection.to_records())

In [7]:
# records_and_molecules

In [8]:
# count = 0 
# for td_record, molecule in tqdm.tqdm(records_and_molecules):
#     # take only the optimized grid points
#     for opt in td_record.minimum_optimizations.values():
#         last = opt.trajectory[-1] #qc_client.get_records(record_ids=[opt.trajectory_ids_[-1]])
#         last_mol = last.molecule
#         mapped_smiles = last_mol.identifiers.canonical_isomeric_explicit_hydrogen_mapped_smiles
#         coords = last_mol.geometry * bohr_to_angstrom
#         energy = last.properties["return_energy"] * hartree_to_kcal
#         gradient = np.array(last.properties["scf total gradient"]).reshape((-1, 3))
#         forces = ((-gradient) * hartree_to_kcal / bohr_to_angstrom)
#         entry = {
#             "coords": coords,
#             "energy": energy,
#             "forces": forces,
#         }
#         data_by_smiles[mapped_smiles].append(entry)

#     count += 1
#     if count > 2:
#         break

In [9]:
# # convert to smee's expected format
# descent_entries = []
# for mapped_smiles, entries in data_by_smiles.items():
#     entry = {
#         "smiles": mapped_smiles,
#         "coords": torch.tensor([x["coords"] for x in entries]),
#         "energy": torch.tensor([x["energy"] for x in entries]),
#         "forces": torch.tensor([x["forces"] for x in entries]),
#     }
#     descent_entries.append(entry)

In [10]:
# # this dataset can get downloaded, processed once, saved and reused
# dataset = descent.targets.energy.create_dataset(entries=descent_entries)
# dataset.save_to_disk("test-smee-data")

## Assign parameters to molecules in the dataset

In [11]:
from openff.toolkit import Molecule, ForceField
import tqdm
import smee.converters
from pydantic import Field

In [12]:
# uncomment if reloading data
import datasets

dataset = datasets.Dataset.load_from_disk("test-smee-data")

In [13]:
# reformat dataset lists to torch tensors
dataset.set_format('torch', columns=['energy', 'coords','forces'], output_all_columns=True)

In [14]:
# this is what a single entry looks like
dataset[0]

{'coords': tensor([-2.7269, -1.4295, -0.0181,  ..., -0.1040,  0.5878, -1.0884]),
 'energy': tensor([-264210.7812, -264210.3438, -264209.5312, -264208.5312, -264207.7188,
         -264208.7812, -264210.3438, -264211.3125, -264212.4062, -264214.4375,
         -264217.4375, -264220.2812, -264222.5000, -264211.6250, -264212.4688,
         -264212.9688, -264213.6250, -264215.5000, -264218.2500, -264223.8438,
         -264224.3750, -264224.1562, -264223.0625, -264221.0625]),
 'forces': tensor([ 0.4746, -0.5829, -1.1677,  ..., -0.0039,  0.0169,  0.0220]),
 'smiles': '[H:6][C@:5]1([C:7]([C:8]([C:9]([N+:10]1([H:21])[H:22])([H:19])[H:20])([H:17])[H:18])([H:15])[H:16])[C:2](=[O:1])[N:3]([H:11])[C:4]([H:12])([H:13])[H:14]'}

Below we specify a starting force field.
Normally we would initialize parameters using the Modified Seminario method,
[example here](https://github.com/openforcefield/sage-2.2.1/blob/main/03_generate-initial-ff/create-msm-ff.py),
but here we just start from Sage 2.2.1.

[Josh's repo](https://github.com/jthorton/SPICE-SMEE/blob/main/fit-v1/training/001-expand_torsions.py)
has examples on expanding torsions too.

In [15]:
# starting_ff = ForceField("lee-krimm-force-field.offxml", load_plugins = True)

In [16]:
import torch
import smee

# @smee.potentials.potential_energy_fn("LeeKrimm", "leekrimm")
# def compute_leekrimm_energy(
#     system: smee.TensorSystem,
#     potential: smee.TensorPotential,
#     conformer: torch.Tensor,
# ) -> torch.Tensor:
#     if conformer.ndim == 2:
#         conformer = conformer.unsqueeze(0)

#     particle_idxs = smee.potentials.broadcast_idxs(system, potential)
#     n_matches = particle_idxs.shape[0]

#     if n_matches == 0:
#         return torch.zeros(
#             (conformer.shape[0],),
#             dtype=conformer.dtype,
#             device=conformer.device
#         )

#     parameters = smee.potentials.broadcast_parameters(system, potential)

#     v2 = parameters[:, potential.parameter_cols.index("V2")]
#     v4 = parameters[:, potential.parameter_cols.index("V4")]
#     t  = parameters[:, potential.parameter_cols.index("t")]
#     s  = parameters[:, potential.parameter_cols.index("s")]

#     conf_idx = torch.arange(conformer.shape[0], device=conformer.device)[:, None]

#     a = conformer[conf_idx, particle_idxs[:, 0]]
#     b = conformer[conf_idx, particle_idxs[:, 1]]
#     c = conformer[conf_idx, particle_idxs[:, 2]]
#     d = conformer[conf_idx, particle_idxs[:, 3]]

#     bc = b - c
#     dc = d - c
#     normal_vector = torch.cross(bc, dc, dim=-1)
#     oop_vector = a - b

#     cos_theta = torch.sum(oop_vector * normal_vector, dim=-1) / (
#         torch.norm(oop_vector, dim=-1) * torch.norm(normal_vector, dim=-1)
#     )
#     cos_theta = torch.clamp(cos_theta, -1.0, 1.0)
#     h = torch.acos(cos_theta)

#     h_abs = torch.abs(h)
#     denom = 1.0 - torch.pow(h_abs, s)
#     safe_denom = torch.where(denom == 0, torch.tensor(1e-8, device=denom.device), denom)

#     frac = torch.pow(h_abs, t) / safe_denom
#     energy = (v2 * frac**2) + (v4 * frac**4)

#     energy = energy.sum(dim=-1)

#     return energy

In [17]:
from smirnoff_plugins.collections.valence import SMIRNOFFLeeKrimmCollection
from smee import TensorPotential, ValenceParameterMap
import smee.converters
import torch
import openff.toolkit
import openff.units

# def convert_leekrimm_handlers(
#     handlers: list[SMIRNOFFLeeKrimmCollection],
#     topologies: list[openff.toolkit.Topology],
# ) -> tuple[TensorPotential, list[ValenceParameterMap]]:
    
#     potential = smee.converters.openff._openff._handlers_to_potential(
#         handlers,
#         "LeeKrimm",
#         ("V2", "V4", "t", "s"),
#         attribute_cols=(),
#     )
#     potential.fn = "leekrimm"

#     parameter_key_to_idx = {param_key: i for i, param_key in enumerate(potential.parameter_keys)}
#     parameter_maps = []

#     for handler, topology in zip(handlers, topologies, strict=True):
#         assignment_map = {}

#         for key, param_key in handler.key_map.items():
#             indices = tuple(key.atom_indices)
#             assignment_map[indices] = parameter_key_to_idx[param_key]

#         assignment_matrix = torch.zeros(
#             (len(assignment_map), len(potential.parameters)), dtype=torch.float64
#         )
#         for torsion_idx, (atom_indices, parameter_idx) in enumerate(assignment_map.items()):
#             assignment_matrix[torsion_idx, parameter_idx] = 1.0

#         parameter_map = BondedParameterMap(
#             particle_idxs=torch.tensor(list(assignment_map.keys()), dtype=torch.int64),
#             assignment_matrix=assignment_matrix.to_sparse()
#         )

#         parameter_maps.append(parameter_map)

#     return potential, parameter_maps

# KCAL_PER_MOL = openff.units.unit.kilocalories / openff.units.unit.mole
# RADIAN = openff.units.unit.radian
# UNITLESS = openff.units.unit.dimensionless

# @smee.converters.smirnoff_parameter_converter(
#     "LeeKrimm",
#     {
#         "V2": KCAL_PER_MOL,
#         "V4": KCAL_PER_MOL,
#         "t": UNITLESS,
#         "s": UNITLESS,
#     },
# )
# def convert_leekrimm(
#     handlers: list[SMIRNOFFLeeKrimmCollection],
#     topologies: list[openff.toolkit.Topology],
# ) -> tuple[TensorPotential, list[ValenceParameterMap]]:
#     return convert_leekrimm_handlers(handlers, topologies)



In [18]:
# smee.potentials.potential_energy_fn("LeeKrimm", "leekrimm")(compute_leekrimm_energy);


In [19]:
from openff.toolkit import ForceField, Molecule
import openff.interchange as interchange
import torch
import openff.units

leekrimm_force_field = ForceField("lee_krimm.offxml", load_plugins=True)
leekrimm_force_field.deregister_parameter_handler("ImproperTorsions")

leekrimm_handler = leekrimm_force_field.get_parameter_handler("LeeKrimm")

molecule = Molecule.from_smiles("c1n(CCO)c(C(F)(F)(F))cc1CNCCl")
molecule.generate_conformers(n_conformers=1)

conformer = torch.tensor(molecule.conformers[0].m_as(openff.units.unit.angstrom))

leekrimm_interchange = interchange.Interchange.from_smirnoff(
    leekrimm_force_field, molecule.to_topology()
)

leekrimm_interchange


Interchange with 7 collections, non-periodic topology with 28 atoms.

In [20]:
from openff.toolkit import ForceField, Molecule, Topology

topology = Topology.from_molecules([molecule])

molecule_force_list = leekrimm_force_field.label_molecules(topology)

for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f"Forces for molecule {mol_idx}")
    for force_tag, force_dict in mol_forces.items():
        print(f"\n{force_tag}:")
        for atom_indices, parameter in force_dict.items():
            atomstr = ""
            for idx in atom_indices:
                atomstr += f"{idx:>3}"
            print(
                f"atoms: {atomstr}  parameter_id: {parameter.id}  smirks {parameter.smirks}"
            )



Forces for molecule 0

Constraints:

Bonds:
atoms:   0  1  parameter_id: b8  smirks [#6X3:1]-[#7X3:2]
atoms:   0 11  parameter_id: b6  smirks [#6X3:1]=[#6X3:2]
atoms:   0 16  parameter_id: b85  smirks [#6X3:1]-[#1:2]
atoms:   1  2  parameter_id: b7  smirks [#6:1]-[#7:2]
atoms:   1  5  parameter_id: b8  smirks [#6X3:1]-[#7X3:2]
atoms:   2  3  parameter_id: b1  smirks [#6X4:1]-[#6X4:2]
atoms:   2 17  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   2 18  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   3  4  parameter_id: b14  smirks [#6:1]-[#8:2]
atoms:   3 19  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   3 20  parameter_id: b84  smirks [#6X4:1]-[#1:2]
atoms:   4 21  parameter_id: b88  smirks [#8:1]-[#1:2]
atoms:   5  6  parameter_id: b2  smirks [#6X4:1]-[#6X3:2]
atoms:   5 10  parameter_id: b6  smirks [#6X3:1]=[#6X3:2]
atoms:   6  7  parameter_id: b69  smirks [#6X4:1]-[#9:2]
atoms:   6  8  parameter_id: b69  smirks [#6X4:1]-[#9:2]
atoms:   6  9  parameter_id: b69  smirks [#6X

In [21]:
leekrimm_tensor_ff, [leekrimm_topology] = smee.converters.convert_interchange(leekrimm_interchange)


In [22]:
all_smiles = []
interchanges = []
for entry in tqdm.tqdm(dataset):
    mol = Molecule.from_mapped_smiles(
        entry["smiles"],
        allow_undefined_stereo=True
    )
    all_smiles.append(entry["smiles"])
    interchange = leekrimm_force_field.create_interchange(mol.to_topology())
    interchanges.append(interchange)
    
smee_force_field, smee_topologies = smee.converters.convert_interchange(interchanges)
topologies = dict(zip(all_smiles, smee_topologies))

100%|███████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.06it/s]


## Fit

Now we can set up and run the fit.

In [23]:
import descent.train
import descent.targets.energy

import math
import pathlib
import tensorboardX
import more_itertools


In [24]:
# specify which parameters to train
# and some details about them
# they're scaled so they're roughly on the same order of magnitude

parameters = {
    "Bonds": descent.train.ParameterConfig(
        cols=["k", "length"],
        scales={"k": 1e-2, "length": 1.0}, # normalize so roughly equal
        limits={"k":[0.0, None], "length": [0.0, None]}
        # the include/exclude types are Interchange PotentialKey.id's -- typically SMIRKS
        # include=[], <-- bonds to train. Not specifying trains all
        # exclude=[], <-- bonds NOT to train
    ),
    "Angles": descent.train.ParameterConfig(
        cols=["k", "angle"],
        scales={"k": 1e-2, "angle": 1.0},
        limits={"k": [0.0, None], "angle": [0.0, math.pi]}
    ),
    "ProperTorsions": descent.train.ParameterConfig(
        # fit ks
        cols=["k"],
        scales={"k": 1.0},
    ),
    "LeeKrimm": descent.train.ParameterConfig(
        cols=["V2", "V4", "t", "s"],
        scales={"V2": 1.0, "V4": 1.0},
        limits={"V2": [0.0, None], "V4": [0.0, None], "t": [0.0, None], "s": [0.0, None]}
    )


}

In [25]:
trainable = descent.train.Trainable(
    force_field=smee_force_field,
    parameters=parameters,
    attributes={}
)

In [26]:
print(leekrimm_force_field.registered_parameter_handlers)

['Constraints', 'Bonds', 'Angles', 'ProperTorsions', 'vdW', 'Electrostatics', 'LibraryCharges', 'ToolkitAM1BCC', 'LeeKrimm']


In [27]:
# optional below if you want cool tensorboard logging
def write_metrics(
        epoch: int,
        loss: torch.Tensor,
        loss_energy: torch.Tensor,
        loss_forces: torch.Tensor,
        writer: tensorboardX.SummaryWriter
):
    print(f"epoch={epoch} loss={loss.detach().item():.6f}", flush=True)

    writer.add_scalar("loss", loss.detach().item(), epoch)
    writer.add_scalar("loss_energy", loss_energy.detach().item(), epoch)
    writer.add_scalar("loss_forces", loss_forces.detach().item(), epoch)

    writer.add_scalar("rmse_energy", math.sqrt(loss_energy.detach().item()), epoch)
    writer.add_scalar("rmse_forces", math.sqrt(loss_forces.detach().item()), epoch)
    writer.flush()

Specify some hyperparameters, n_epochs is intentionally very low to guarantee fast execution.

In [28]:
N_EPOCHS = 10
LEARNING_RATE = 0.01
BATCH_SIZE = 500

In [29]:
# make directory to save files in
directory = pathlib.Path("my-smee-fit")
directory.mkdir(exist_ok=True, parents=True)


Run fit below.

In [30]:
# load tensorboard extension so we can view in notebook
%load_ext tensorboard

In [31]:
trainable_parameters = trainable.to_values()
device = trainable_parameters.device.type

torch.autograd.set_detect_anomaly(True)

with tensorboardX.SummaryWriter(str(directory)) as writer:
    optimizer = torch.optim.Adam([trainable_parameters], lr=LEARNING_RATE, amsgrad=True)
    dataset_indices = list(range(len(dataset)))

    for i in range(N_EPOCHS):
        ff = trainable.to_force_field(trainable_parameters)
        total_loss = torch.zeros(size=(1,), device=device)
        energy_loss = torch.zeros(size=(1,), device=device)
        force_loss = torch.zeros(size=(1,), device=device)
        grad = None
    
        for batch_ids in tqdm.tqdm(
            more_itertools.batched(dataset_indices, BATCH_SIZE),
            desc='Calculating energies',
            ncols=80, total=math.ceil(len(dataset) / BATCH_SIZE)
        ):
            batch = dataset.select(indices=batch_ids)
            true_batch_size = len(dataset)
            batch_configs = sum([len(d["energy"]) for d in batch])

            e_ref, e_pred, f_ref, f_pred = descent.targets.energy.predict(
                batch, ff, topologies, "mean"
            )   
            # L2 loss
            batch_loss_energy = ((e_pred - e_ref) ** 2).sum() / true_batch_size
            batch_loss_force = ((f_pred - f_ref) ** 2).sum() / true_batch_size

            # Equal sum of L2 loss on energies and forces
            batch_loss = batch_loss_energy + batch_loss_force

            (batch_grad, ) = torch.autograd.grad(batch_loss, trainable_parameters, create_graph=True)
            batch_grad = batch_grad.detach()
            if grad is None:
                grad = batch_grad
            else:
                grad += batch_grad
            
            # keep sum of squares to report MSE at the end
            total_loss += batch_loss.detach()
            energy_loss += batch_loss_energy.detach()
            force_loss += batch_loss_force.detach()
        
        trainable_parameters.grad = grad
        
        write_metrics(
            epoch=i, loss=total_loss, loss_energy=energy_loss,
            loss_forces=force_loss, writer=writer
        )

        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
            torch.save(
                trainable.to_force_field(trainable_parameters),
                directory / f"force-field-epoch-{i}.pt"
            )

    torch.save(
        trainable.to_force_field(trainable_parameters),
        directory / "final-force-field.pt"
    )
    

Calculating energies: 100%|███████████████████████| 1/1 [00:02<00:00,  2.49s/it]

epoch=0 loss=1849424.250000



Calculating energies: 100%|███████████████████████| 1/1 [00:02<00:00,  2.40s/it]

epoch=1 loss=1670482.000000



Calculating energies: 100%|███████████████████████| 1/1 [00:02<00:00,  2.36s/it]

...

epoch=7 loss=935493.812500



Calculating energies: 100%|███████████████████████| 1/1 [00:02<00:00,  2.49s/it]

epoch=8 loss=855176.125000



Calculating energies: 100%|███████████████████████| 1/1 [00:02<00:00,  2.47s/it]

epoch=9 loss=783553.750000


Metrics can be viewed in tensorboard below.

`tensorboard --logdir my-smee-fit` can also be run on command line instead of in the notebook.

In [32]:
%tensorboard --logdir my-smee-fit

## Convert back to OFFXML

In [33]:
for potential in smee_force_field.potentials:
    handler_name = potential.parameter_keys[0].associated_handler

    parameter_attrs = potential.parameter_cols
    parameter_units = potential.parameter_units

    if handler_name in ["Bonds", "Angles"]:
        handler = leekrimm_force_field.get_parameter_handler(handler_name)
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            for j, (p, unit) in enumerate(zip(parameter_attrs, parameter_units)):
                setattr(ff_parameter, p, opt_parameters[j] * unit)

    elif handler_name in ["ProperTorsions"]:
        handler = leekrimm_force_field.get_parameter_handler(handler_name)
        k_index = parameter_attrs.index('k')
        p_index = parameter_attrs.index('periodicity')
        # we need to collect the k values into a list across the entries
        collection_data = defaultdict(dict)
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            # find k and the periodicity
            k = opt_parameters[k_index] * parameter_units[k_index]
            p = int(opt_parameters[p_index])
            collection_data[smirks][p] = k
        # now update the force field
        for smirks, k_s in collection_data.items():
            ff_parameter = handler[smirks]
            k_mapped_to_p = [k_s[p] for p in ff_parameter.periodicity]
            ff_parameter.k = k_mapped_to_p

    elif handler_name in ["ImproperTorsions"]:
        k_index = parameter_attrs.index('k')
        handler = leekrimm_force_field.get_parameter_handler(handler_name)
        # we only fit the v2 terms for improper torsions so convert to list and set
        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()
            ff_parameter.k = [opt_parameters[k_index] * parameter_units[k_index]]

    elif handler_name == "LeeKrimm":
        handler = leekrimm_force_field.get_parameter_handler(handler_name)
        param_indices = {
            name: i for i, name in enumerate(parameter_attrs)
        }

        for i, opt_parameters in enumerate(potential.parameters):
            smirks = potential.parameter_keys[i].id
            ff_parameter = handler[smirks]
            opt_parameters = opt_parameters.detach().cpu().numpy()

            for param_name, unit in zip(parameter_attrs, parameter_units):
                value = opt_parameters[param_indices[param_name]] * unit
                setattr(ff_parameter, param_name, value)


leekrimm_force_field.to_file("lee_krimm_final-force-field.offxml")